In [7]:
import numpy as np
import os
import random
import sys
import tensorflow as tf
from tensorflow.contrib.distributions import MultivariateNormalFullCovariance
import vrep
import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import BatchNormalization, Dense, Input
from keras.callbacks import TensorBoard
from keras import backend as K
from common import *



In [ ]:
K

## policyLoss(actions, advantage)
Idea:  
If advantage is small, probably need to change direction (positive action to negative and vice versa)
Reference:https://medium.com/@awjuliani/simple-reinforcement-learning-with-tensorflow-part-3-model-based-rl-9a6fe0cce99

In [2]:
class GaussianPolicy:
    """
    self.action(6,) is sampled from self.policy, 
    which is a gaussian distribution with mean as the output from self.model
    self.model takes state (24,) and produces an action mean (6,)
    TODO: Constrain self.model output?
    """
    def __init__(self, cov=None, lr=0.05, epochs=1, batch_size=128, seed=0, model_file=None):
        if not model_file:
            self.model = Sequential([
                #Input(shape=(24,), dtype="float32", name="state_l"),
                Dense(256, input_shape=(24,), dtype="float32", kernel_initializer="random_normal", kernel_regularizer=keras.regularizers.l2(0.01), bias_initializer="zeros", activation="relu", name="hidden_l1"),
                BatchNormalization(),
                Dense(64, kernel_initializer="random_normal",  bias_initializer="zeros", kernel_regularizer=keras.regularizers.l2(0.01), activation="relu", name="hidden_l2"),
                BatchNormalization(),
                Dense(6, kernel_initializer="random_normal", kernel_regularizer=keras.regularizers.l2(0.01), bias_initializer="zeros", name="action_l")
            ])
        else:
            # load an existing model
            self.model = load_model(model_file)
        self.trainable_params = self.model.trainable_weights
        print(self.trainable_params)
        self.state = tf.placeholder(dtype=tf.float32, shape=(None, 24))
        self.action_taken = tf.placeholder(dtype=tf.float32, shape=(None, 6))
        self.advantage = tf.placeholder(dtype=tf.float32, shape=(None, 1))
        self.action_mean = self.model(self.state)
        self.action_cov = tf.constant(np.eye(6).astype("float32")) if cov==None else tf.constant(cov)
        self.action_dist = MultivariateNormalFullCovariance(self.action_mean, self.action_cov)
        self.action_probability = self.action_dist.prob(self.action_taken)
        self.action_sample =self.action_dist.sample()
        #self.loglike = tf.log(self.action_probability)
        self.loglike = -0.5 * tf.matmul(tf.square(self.action_taken - self.action_mean), tf.matrix_inverse(self.action_cov))
        self.loss = -tf.reduce_mean(self.loglike * self.advantage)
        #self.loss = -tf.reduce_mean(self.loglike)
        self.params_updates = tf.train.GradientDescentOptimizer(lr).minimize(self.loss, var_list=self.trainable_params)
        
        self.epochs = epochs
        self.batch_size = batch_size
        self.seed = seed
        self.session = tf.Session()

        for param in self.trainable_params:         
            #print(param.initializer)
            self.session.run(param.initializer)
            self.session.run(tf.global_variables_initializer())
  
    def __del__(self):
        self.session.close()
    
    def sampleAction(self, state):
        return self.session.run(self.action_sample, feed_dict={self.state:state, K.learning_phase():0})

    def train(self, state, action_taken, advantage):
        # Gradient ascent
        #params_updates = []
        #self.session.run(tf.global_variables_initializer())
        #batch_size = self.session.run(tf.shape(self.loglike)[0], feed_dict={self.action_taken:action_taken, self.state:state})
        #print(batch_size)
        for train_step in range(self.epochs):
            np.random.seed(self.seed)
            np.random.shuffle(action_taken)
            np.random.seed(self.seed)
            np.random.shuffle(state)
            np.random.seed(self.seed)
            np.random.shuffle(advantage)
            
            self.session.run(self.params_updates, feed_dict={self.action_taken:action_taken, self.state:state, self.advantage:advantage
                                                            , K.learning_phase():1})
            print("== Epoch: %d =="%(train_step))
            #if train_step % 10 == 0:
            loglike_vec, loss, action_prob = self.session.run([self.loglike, self.loss, self.action_probability], feed_dict={self.action_taken:action_taken, self.state:state, self.advantage:advantage
                                                                                    , K.learning_phase():0})
            print("Log likelihood")
            print(loglike_vec)
            print("Loss")
            print(loss)
            #print("Action taken")
            #print(action_taken)
            print("Action prob")
            print(action_prob)
            print("Advantage")
            #print(advantage)

    def save(self, model_file):
        """
        Save the internal model to location model_file
        """
        self.model.save(model_file)

In [3]:
def getModel():
    """
    Model for transition dynamics, rewards and termination of episodes.
    Inputs: [PrevState(24,), Action(6,)](30,)
    Outputs: [NestState-PrevState(24,), Reward](25,)
    """
    prevState_action_l = Input(shape=(30,), dtype="float32", name="prevState_action_l")
    H_l = Dense(256, kernel_initializer="normal", activation="relu", name="hidden_l1")(prevState_action_l)
    H_l = BatchNormalization()(H_l)
    H_l = Dense(64, kernel_initializer="normal", activation="relu", name="hidden_l2")(H_l)
    H_l = BatchNormalization()(H_l)
    nextState_reward_l = Dense(25, kernel_initializer="normal", name="nextState_l")(H_l)
    #dest_l = Dense(1, kernel_initializer="normal", activation="sigmoid", name="dest_l")(H_l)
    model = Model(inputs=prevState_action_l, outputs=nextState_reward_l)
    model.compile(loss="mse", optimizer="rmsprop")
    return model

def getPolicy():
    return GaussianPolicy()


def getAdvantages(rewards, discount_factor):
    eps = rewards.shape[0]
    advantages = np.zeros_like(rewards)
    running_discounted_advantages = 0
    for i in range(eps - 1, -1, -1):
        running_discounted_advantages = running_discounted_advantages * discount_factor + rewards[i]
        advantages[i] = running_discounted_advantages
    return advantages

In [4]:
class VREPPushTaskEnvironment():
    MAX_JOINT_VELOCITY = 1.0
    INITIAL_JOINT_POSITIONS = [np.pi, 1.5 * np.pi, 1.5 * np.pi, np.pi, np.pi, np.pi]
    INITIAL_CUBOID_POSITION = [0., 0.5, 0.05]
    
    def __init__(self):
        vrep.simxFinish(-1) # just in case, close all opened connections
        self.client_ID=vrep.simxStart('127.0.0.1',19997,True,True,5000,5) # Connect to V-REP
        
         # enable the synchronous mode on the client:
        vrep.simxSynchronous(self.client_ID,True)
        # start the simulation:
        vrep.simxStartSimulation(self.client_ID, vrep.simx_opmode_blocking)

        
    def __del__(self):
        # stop the simulation:
        vrep.simxStopSimulation(self.client_ID, vrep.simx_opmode_blocking)
        # before closing the connection to V-REP, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
        vrep.simxGetPingTime(self.client_ID)
        # disconnect
        vrep.simxFinish(self.client_ID)
        
        
    def __enter__(self):
        return self
        
    def __exit__(self, exception_type, exception_value, traceback):
        del self
        
    def reset(self, is_first_reset):
        """
        Reset the environment
        Return initial state
        
        """
        
        if not is_first_reset:
            # tear down datastreams
            for i in range(6):
                _, _ = vrep.simxGetObjectFloatParameter(self.client_ID, self.joint_handles[i], 2012, vrep.simx_opmode_discontinue)
                _, _ = vrep.simxGetJointPosition(self.client_ID, self.joint_handles[i],
                        vrep.simx_opmode_discontinue)
            _, _ = vrep.simxGetObjectPosition(self.client_ID, self.gripper_handle, -1, vrep.simx_opmode_discontinue)
            _, _ = vrep.simxGetObjectOrientation(self.client_ID, self.gripper_handle, -1, vrep.simx_opmode_discontinue)
            _, _ = vrep.simxGetObjectPosition(self.client_ID, self.cuboid_handle, -1, vrep.simx_opmode_discontinue)
            _, _ = vrep.simxGetObjectPosition(self.client_ID, self.target_plane_handle, -1, vrep.simx_opmode_discontinue)

            # remove Mico
            vrep.simxRemoveModel(self.client_ID, self.model_base_handle, vrep.simx_opmode_blocking)
        
        # get handles
        _, self.cuboid_handle = vrep.simxGetObjectHandle(self.client_ID, 'Cuboid', vrep.simx_opmode_blocking)
        _, self.target_plane_handle = vrep.simxGetObjectHandle(self.client_ID, 'TargetPlane', vrep.simx_opmode_blocking)
        
        _, self.model_base_handle = vrep.simxLoadModel(self.client_ID, 'models/robots/non-mobile/MicoRobot.ttm', 0, vrep.simx_opmode_blocking)
        self.joint_handles = [-1, -1, -1, -1, -1, -1]
        for i in range(6):
            _, self.joint_handles[i] = vrep.simxGetObjectHandle(self.client_ID, 'Mico_joint' + str(i+1), vrep.simx_opmode_blocking)
        _, self.gripper_handle = vrep.simxGetObjectHandle(self.client_ID, 'MicoHand', vrep.simx_opmode_blocking)

        # initialise mico joint positions, cuboid orientation and cuboid position
        vrep.simxPauseCommunication(self.client_ID, 1)
        for i in range(6):
            vrep.simxSetJointPosition(self.client_ID, self.joint_handles[i], VREPPushTaskEnvironment.INITIAL_JOINT_POSITIONS[i], vrep.simx_opmode_oneshot)
        vrep.simxSetObjectOrientation(self.client_ID, self.cuboid_handle, -1, [0, 0, 0], vrep.simx_opmode_oneshot)
        vrep.simxSetObjectPosition(self.client_ID, self.cuboid_handle, -1, VREPPushTaskEnvironment.INITIAL_CUBOID_POSITION, vrep.simx_opmode_oneshot)
        vrep.simxPauseCommunication(self.client_ID, 0)
        vrep.simxGetPingTime(self.client_ID)

        current_vel = np.array([0, 0, 0, 0, 0, 0], dtype='float')
        joint_angles = np.array([0, 0, 0, 0, 0, 0], dtype='float')

        # set up datastreams
        for i in range(6):
            _, current_vel[i] = vrep.simxGetObjectFloatParameter(self.client_ID, self.joint_handles[i], 2012,
                    vrep.simx_opmode_streaming)
            _, joint_angles[i] = vrep.simxGetJointPosition(self.client_ID, self.joint_handles[i], vrep.simx_opmode_streaming)
        _, gripper_pos = vrep.simxGetObjectPosition(self.client_ID, self.gripper_handle, -1, vrep.simx_opmode_streaming)
        _, gripper_orient = vrep.simxGetObjectOrientation(self.client_ID, self.gripper_handle, -1, vrep.simx_opmode_streaming)
        _, cuboid_pos = vrep.simxGetObjectPosition(self.client_ID, self.cuboid_handle, -1, vrep.simx_opmode_streaming)
        _, target_plane_pos = vrep.simxGetObjectPosition(self.client_ID, self.target_plane_handle, -1, vrep.simx_opmode_streaming)

        # destroy dummy arrays for setting up the datastream
        del current_vel, joint_angles, gripper_pos, gripper_orient, cuboid_pos, target_plane_pos

        # obtain first state
        current_state = getCurrentState(self.client_ID, self.joint_handles, self.gripper_handle, self.cuboid_handle,
                self.target_plane_handle)
        
        self.state = current_state
        
        return current_state
    
    def getRewards(self, state):
        """
            Return the sum of the Euclidean distance between gripper and cuboid and the Euclidean distance between cuboid and targetPlane.
        """
        return -np.sqrt(np.sum(np.square(state[-6:-3]))) + np.sqrt(np.sum(np.square(state[-3:])))

    
    def getCurrentState(self, client_ID, joint_handles, gripper_handle, cuboid_handle, target_plane_handle):
        """
            TODO: Refactor away arguments
            Return the state as an array of shape (24, )
            [current_vel, joint_angles, gripper_pos, gripper_orient, cuboid_gripper_vec, target_plane_cuboid_vec]
             6              6               3           3               3                   3
        """
        current_vel = np.array([0, 0, 0, 0, 0, 0], dtype='float')
        joint_angles = np.array([0, 0, 0, 0, 0, 0], dtype='float')
        # obtain first state
        for i in range(6):
            ret, current_vel[i] = vrep.simxGetObjectFloatParameter(client_ID, joint_handles[i], 2012,
                    vrep.simx_opmode_buffer)
            while ret != vrep.simx_return_ok:
                ret, current_vel[i] = vrep.simxGetObjectFloatParameter(client_ID, joint_handles[i], 2012,
                        vrep.simx_opmode_buffer)
            ret, joint_angles[i] = vrep.simxGetJointPosition(client_ID, joint_handles[i], vrep.simx_opmode_buffer)
            while ret != vrep.simx_return_ok:
                ret, joint_angles[i] = vrep.simxGetJointPosition(client_ID, joint_handles[i], vrep.simx_opmode_buffer)
        ret, gripper_pos = vrep.simxGetObjectPosition(client_ID, gripper_handle, -1, vrep.simx_opmode_buffer)
        while ret != vrep.simx_return_ok:
            ret, gripper_pos = vrep.simxGetObjectPosition(client_ID, gripper_handle, -1, vrep.simx_opmode_buffer)
        ret, gripper_orient = vrep.simxGetObjectOrientation(client_ID, gripper_handle, -1, vrep.simx_opmode_buffer)
        while ret != vrep.simx_return_ok:
            ret, gripper_orient = vrep.simxGetObjectOrientation(client_ID, gripper_handle, -1, vrep.simx_opmode_buffer)
        gripper_pos = np.array(gripper_pos)
        gripper_orient = np.array(gripper_orient)

        ret, cuboid_pos = vrep.simxGetObjectPosition(client_ID, cuboid_handle, -1, vrep.simx_opmode_buffer)
        while ret != vrep.simx_return_ok:
            ret, cuboid_pos = vrep.simxGetObjectPosition(client_ID, cuboid_handle, -1, vrep.simx_opmode_buffer)
        cuboid_pos = np.array(cuboid_pos)

        ret, target_plane_pos = vrep.simxGetObjectPosition(client_ID, target_plane_handle, -1, vrep.simx_opmode_buffer)
        while ret != vrep.simx_return_ok:
            ret, target_plane_pos = vrep.simxGetObjectPosition(client_ID, target_plane_handle, -1, vrep.simx_opmode_buffer)
        target_plane_pos = np.array(target_plane_pos)

        cuboid_gripper_vec = cuboid_pos - gripper_pos
        target_plane_cuboid_vec = target_plane_pos - cuboid_pos

        return np.concatenate([current_vel, joint_angles, gripper_pos, gripper_orient, cuboid_gripper_vec,
            target_plane_cuboid_vec])

    
    def step(self, actions):
        """
        Execute sequences of actions (None, 6) in the environment
        Return sequences of subsequent states and rewards
        """
        next_states = []
        rewards = []
        for i in range(actions.shape[0]):
            current_vel = self.state[:6] + actions[i, :]
            vrep.simxPauseCommunication(self.client_ID, 1)
            for i in range(6):
                vrep.simxSetJointTargetVelocity(self.client_ID, self.joint_handles[i], current_vel[i],
                        vrep.simx_opmode_oneshot)
            vrep.simxPauseCommunication(self.client_ID, 0)
            vrep.simxSynchronousTrigger(self.client_ID)
            vrep.simxSynchronousTrigger(self.client_ID)
            vrep.simxSynchronousTrigger(self.client_ID)
            vrep.simxSynchronousTrigger(self.client_ID)
            vrep.simxSynchronousTrigger(self.client_ID)
            # make sure all commands are exeucted
            vrep.simxGetPingTime(self.client_ID)
            # obtain next state
            next_state = getCurrentState(self.client_ID, self.joint_handles, self.gripper_handle, self.cuboid_handle,
                    self.target_plane_handle)
            next_states.append(next_state)
            rewards.append(self.getRewards(next_state))

        next_states = np.concatenate(next_states)
        rewards = np.array(rewards)
        return next_states, rewards
    
        

In [ ]:
a = np.array([])
print(a is not None)

In [18]:
GREEDY_FACTOR = 0.9 # Fraction of times to choose greedy actions TODO: Increase over time

EPS_LENGTH = 200
NUM_EPS = 150

NEW_MODEL = False
MODEL_FILE = "model/PolicyGradient/model01.h5"

NEW_POLICY = True
POLICY_FILE = "model/PolicyGradient/policy02.h5"
SEED = 10

MONITOR_LOG_FILE = "" 

DISCOUNT_FACTOR = 0.9

EPSILON = 0.1

BATCH_SIZE = 128
EPOCHS = 20

random.seed(SEED)

with VREPPushTaskEnvironment() as env:
    model = getModel() if NEW_MODEL else load_model(MODEL_FILE)
    policy = getPolicy() if NEW_POLICY else GaussianPolicy(model_file=POLICY_FILE)
    # obtain first state
    current_state = env.reset(True)
    Xs_mean = None
    Xs_std = None
    ys_mean = None
    ys_std = None
    
    policy_Xs_mean = None
    policy_Xs_std = None
    policy_ys_mean = None
    policy_ys_std = None
    policy_rs_mean = None
    policy_rs_std = None
    for i in range(NUM_EPS):
        print("{}th episode".format(i))
        states = [current_state[np.newaxis, :]]
        actions = []
        rewards = []
        GREEDY_FACTOR = 0.5 if i < 100 else 0.9
        # collect trajectory
        for step in range(EPS_LENGTH):
            # select greedy action according to GREEDY_FACTOR
            if (not NEW_POLICY) or (np.random.rand() < GREEDY_FACTOR and policy_Xs_mean is not None):
                #action = policy.sampleAction(invStandardise(current_state[np.newaxis, :], policy_Xs_mean, policy_Xs_std))
                #action = invStandardise(action, policy_ys_mean, policy_ys_std)
                action = policy.sampleAction(current_state[np.newaxis, :])
            else:
                action = generateRandomVel(MAX_JOINT_VELOCITY)[np.newaxis, :]
            next_state, reward = env.step(action)
           
            actions.append(action)
            states.append(next_state[np.newaxis, :])
            rewards.append(reward)
            # proceed to next state
            current_state = next_state[:]
            
        states = np.concatenate(states, axis=0)
        actions = np.concatenate(actions, axis=0)
        rewards = np.array(rewards)
        advantages = getAdvantages(rewards, DISCOUNT_FACTOR)
        if NEW_MODEL:
            # X = [current_states(,24), actions(,6)]
            # y = [next_states - current_states(,24), rewards]
            X = np.concatenate([states[:-1, :], actions], axis=1)
            y = np.concatenate([states[1:, :] - states[:-1, :], rewards], axis=1)
            # standardise training data
            if Xs_mean is None:
                Xs_mean = np.mean(X, axis=0)
                Xs_std = np.std(X, axis=0)
                ys_mean = np.mean(y, axis=0)
                ys_std = np.std(y, axis=0)
            X = standardise(X, Xs_mean, Xs_std)
            y = standardise(y, ys_mean, ys_std)    
            # add zero-mean gaussian noise
            X += np.random.normal(0, 0.05, X.shape)
            y += np.random.normal(0, 0.05, y.shape)

            # train model
            model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2)
        if NEW_POLICY:
            policy_X = states[:-1, :]
            policy_y = actions[:]
            # standardise training data
            """
            if policy_Xs_mean is None:
                policy_Xs_mean = np.mean(policy_X, 0)
                policy_Xs_std = np.std(policy_X, 0)
                policy_ys_mean = np.mean(policy_y, 0)
                policy_ys_std = np.std(policy_y, 0)
                #policy_rs_mean = np.mean(advantages, 0)
                #policy_rs_std = np.std(advantages, 0)
            policy_X = standardise(policy_X, policy_Xs_mean, policy_Xs_std)
            policy_y = standardise(policy_y, policy_ys_mean, policy_ys_std)
            """
            advantages = standardise(advantages, np.mean(advantages), np.std(advantages))

            # train policy
            policy.train(policy_X, policy_y, advantages)
        # Reset the scene
        env.reset(False)
        
    if NEW_MODEL:
        model.save(MODEL_FILE)
    if NEW_POLICY:
        policy.save(POLICY_FILE)

[<tf.Variable 'hidden_l1_11/kernel:0' shape=(24, 256) dtype=float32_ref>, <tf.Variable 'hidden_l1_11/bias:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'batch_normalization_7/gamma:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'batch_normalization_7/beta:0' shape=(256,) dtype=float32_ref>, <tf.Variable 'hidden_l2_11/kernel:0' shape=(256, 64) dtype=float32_ref>, <tf.Variable 'hidden_l2_11/bias:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'batch_normalization_8/gamma:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'batch_normalization_8/beta:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'action_l_5/kernel:0' shape=(64, 6) dtype=float32_ref>, <tf.Variable 'action_l_5/bias:0' shape=(6,) dtype=float32_ref>]
0th episode
== Epoch: 0 ==
Log likelihood
[[ -6.33027077e-01  -1.97710702e-03  -9.12820280e-04  -1.19018387e-02
   -2.30341451e-03  -1.77757849e-03]
 [ -7.82773718e-02  -1.52097018e-02  -3.51491421e-01  -1.96412988e-02
   -3.09578717e-01  -7.14047253e-02]
 [ -5.47033429e-01  -

3th episode
== Epoch: 0 ==
Log likelihood
[[ -1.50495660e-04  -4.23022360e-01  -1.30974531e-01  -2.67477274e-01
   -3.05957973e-01  -6.65186495e-02]
 [ -3.43124479e-01  -1.30678266e-01  -3.91113520e-01  -1.82469022e-02
   -5.04511548e-03  -1.28339007e-02]
 [ -1.98206216e-01  -1.12211946e-02  -2.99180895e-01  -1.95213124e-01
   -2.25535035e-01  -2.17760690e-02]
 ..., 
 [ -1.14028171e-01  -2.87600979e-02  -4.02431279e-01  -4.74252993e-05
   -1.58505049e-02  -6.29887059e-02]
 [ -5.18703982e-02  -1.94632933e-02  -2.52746999e-01  -1.01929568e-01
   -1.39753968e-01  -5.92812411e-02]
 [ -1.80697843e-01  -2.30904624e-01  -8.70618150e-02  -8.19935277e-02
   -4.02981648e-03  -8.43116716e-02]]
Loss
-0.00724819
Action prob
[ 0.00122143  0.00163735  0.00155736  0.00185632  0.00250291  0.00092623
  0.00050621  0.00281833  0.00322584  0.00124615  0.00108931  0.00191659
  0.00159795  0.00166095  0.00162758  0.00133     0.00124676  0.00243821
  0.00125279  0.0011813   0.00103122  0.00093753  0.00146214

6th episode
== Epoch: 0 ==
Log likelihood
[[ -5.27155120e-03  -4.28037882e-01  -1.14686258e-01  -3.19952846e-01
   -3.95744979e-01  -1.92456618e-02]
 [ -9.56076533e-02  -1.29657768e-04  -2.79143639e-03  -1.84620082e-01
   -2.69749284e-01  -3.17330053e-03]
 [ -5.04233360e-01  -9.57325697e-02  -2.33398393e-01  -2.82373396e-04
   -7.52566010e-03  -4.92095910e-02]
 ..., 
 [ -4.28712964e-02  -2.28297576e-01  -2.73725122e-01  -8.94449011e-04
   -2.42123902e-01  -9.50598717e-03]
 [ -1.56355351e-01  -3.09206367e-01  -1.18970042e-02  -1.48361688e-02
   -5.08029275e-02  -3.49392325e-01]
 [ -1.10710740e-01  -1.63330068e-03  -1.09432258e-01  -6.19193278e-02
   -8.49197153e-03  -1.30430430e-01]]
Loss
0.00188604
Action prob
[ 0.0011176   0.00231186  0.0016549   0.00163238  0.00302306  0.00092705
  0.00163084  0.00101911  0.00197793  0.00222261  0.00134897  0.00108197
  0.00177855  0.00175274  0.00084929  0.00152377  0.00230955  0.00095031
  0.00151352  0.00116558  0.00230396  0.00110363  0.00107941 

9th episode
== Epoch: 0 ==
Log likelihood
[[ -6.00259960e-01  -4.52634096e-01  -4.90378216e-02  -1.52093530e-01
   -1.09377911e-03  -5.12631610e-02]
 [ -3.88525754e-01  -1.49171308e-01  -1.31873563e-01  -4.42938507e-01
   -3.99555504e-01  -3.85340393e-01]
 [ -5.80338717e-01  -1.53860211e-01  -1.04308985e-02  -2.32498899e-01
   -7.83169735e-03  -3.09820443e-01]
 ..., 
 [ -7.05734715e-02  -5.76423816e-02  -6.50157928e-02  -7.38709559e-06
   -6.11620490e-03  -1.37909979e-01]
 [ -3.68856796e-04  -1.12715364e-01  -2.47852817e-01  -4.08452600e-02
   -2.71805584e-01  -5.31699993e-02]
 [ -3.80714715e-01  -2.49191970e-02  -2.64729917e-01  -1.37924567e-01
   -6.31700456e-02  -7.44094653e-03]]
Loss
0.00328661
Action prob
[ 0.00109171  0.00060454  0.00110445  0.00106909  0.00121144  0.00104934
  0.00211355  0.00128001  0.00157752  0.00101827  0.00173878  0.00120449
  0.00134565  0.00161239  0.00140606  0.00078115  0.00135974  0.00165742
  0.00114162  0.00181872  0.00110167  0.00140396  0.00254125 

12th episode
== Epoch: 0 ==
Log likelihood
[[ -2.23718147e-04  -1.42538212e-02  -5.37263528e-02  -2.12185588e-02
   -1.09091967e-01  -1.17943145e-01]
 [ -2.50016093e-01  -4.76217508e-01  -8.52967948e-02  -2.84777023e-02
   -2.76316613e-01  -1.08007744e-01]
 [ -3.51430506e-01  -2.68773198e-01  -2.95720133e-03  -2.00234890e-01
   -3.52225080e-02  -3.99917364e-01]
 ..., 
 [ -4.79972363e-01  -5.35405567e-03  -3.02313119e-01  -3.38039994e-01
   -4.15116847e-01  -6.69898018e-02]
 [ -6.77547709e-04  -1.51287613e-03  -2.60620147e-01  -4.31755781e-02
   -2.27115765e-01  -1.06399141e-01]
 [ -3.87238397e-04  -7.63851628e-02  -2.14460909e-01  -3.54887396e-01
   -1.78151876e-01  -2.71531474e-02]]
Loss
-0.00170474
Action prob
[ 0.00293782  0.00118506  0.00114521  0.00122264  0.0017565   0.00099398
  0.00193187  0.00204914  0.00056199  0.00093463  0.00167106  0.0012566
  0.0020011   0.00133866  0.00133774  0.00184537  0.00212454  0.00246786
  0.00187244  0.00150874  0.00272743  0.00157276  0.00110464

...

123th episode
== Epoch: 0 ==
Log likelihood
[[ -4.17215094e+13  -6.76490052e+11  -1.10406423e+15  -2.80353718e+13
   -3.95424788e+14  -1.59282915e+12]
 [ -3.95212615e+13  -5.67424713e+11  -1.02907403e+15  -2.64408819e+13
   -3.68184494e+14  -1.37506626e+12]
 [ -7.18982978e+13  -1.42510378e+12  -1.95576637e+15  -4.86500089e+13
   -7.02055891e+14  -3.20278757e+12]
 ..., 
 [ -6.52879363e+13  -1.24375623e+12  -1.76715745e+15  -4.40665700e+13
   -6.33669341e+14  -2.81770721e+12]
 [ -2.13836984e+13  -1.75362343e+11  -5.21786819e+14  -1.41759412e+13
   -1.86537929e+14  -5.11913624e+11]
 [ -6.95545123e+13  -1.33325403e+12  -1.87942346e+15  -4.70584929e+13
   -6.74228530e+14  -2.97687869e+12]]
Loss
2.29882e+13
Action prob
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

129th episode
== Epoch: 0 ==
Log likelihood
[[ -5.96163900e+12  -1.38526423e+10  -1.14448086e+14  -3.31408946e+12
   -3.96485418e+13  -5.68596767e+10]
 [ -4.37636531e+13  -7.41053170e+11  -1.17570113e+15  -2.96624010e+13
   -4.20199367e+14  -1.77795983e+12]
 [ -2.35808385e+13  -2.44035731e+11  -5.92497181e+14  -1.57323698e+13
   -2.11194112e+14  -6.71506039e+11]
 ..., 
 [ -7.83198873e+12  -1.14122202e+09  -1.66222163e+14  -4.98320749e+12
   -5.84545720e+13  -6.85573980e+10]
 [ -6.04492687e+12  -3.53266425e+06  -1.24855471e+14  -3.83595839e+12
   -4.37902660e+13  -2.41980948e+10]
 [ -1.00042696e+13  -9.92878285e+09  -2.19245783e+14  -6.28147146e+12
   -7.77866141e+13  -1.39250696e+11]]
Loss
-1.58959e+13
Action prob
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

135th episode
== Epoch: 0 ==
Log likelihood
[[ -4.95280277e+13  -6.45309334e+11  -1.43751482e+15  -3.60551473e+13
   -5.22354292e+14  -2.11480071e+12]
 [ -8.96833002e+12  -2.19448832e+08  -2.06067212e+14  -5.77309088e+12
   -7.39853499e+13  -1.59635587e+11]
 [ -4.30729645e+13  -4.82222211e+11  -1.23472619e+15  -3.10834857e+13
   -4.48734995e+14  -1.77138880e+12]
 ..., 
 [ -5.85320114e+13  -8.80033923e+11  -1.72264307e+15  -4.29824347e+13
   -6.26099797e+14  -2.62813123e+12]
 [ -3.59639925e+13  -3.16887958e+11  -1.00820344e+15  -2.56290157e+13
   -3.66238170e+14  -1.34915254e+12]
 [ -1.42805282e+13  -1.51809434e+10  -3.54451370e+14  -9.67036423e+12
   -1.27936716e+14  -3.49280600e+11]]
Loss
1.67042e+13
Action prob
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

141th episode
== Epoch: 0 ==
Log likelihood
[[ -9.10599672e+13  -2.46206326e+12  -2.74016847e+15  -6.73608284e+13
   -9.86447553e+14  -4.99253667e+12]
 [ -7.17035038e+12  -7.40430592e+09  -1.56221097e+14  -4.58415040e+12
   -5.53145608e+13  -6.25153761e+10]
 [ -5.42005915e+13  -7.86700698e+11  -1.54008924e+15  -3.82056425e+13
   -5.56097568e+14  -2.61600497e+12]
 ..., 
 [ -8.59043288e+13  -1.73467173e+12  -2.54444546e+15  -6.21503998e+13
   -9.18431176e+14  -4.72700682e+12]
 [ -5.52055844e+13  -8.22697656e+11  -1.57245880e+15  -3.90022792e+13
   -5.67761055e+14  -2.67987399e+12]
 [ -4.58959199e+13  -9.94698199e+11  -1.32199295e+15  -3.32613382e+13
   -4.74655157e+14  -2.07881175e+12]]
Loss
1.80037e+13
Action prob
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

147th episode
== Epoch: 0 ==
Log likelihood
[[ -3.38623830e+12  -2.03594813e+10  -5.84877699e+13  -1.65764819e+12
   -2.02481787e+13  -1.64817644e+10]
 [ -3.16891139e+13  -2.92454007e+11  -8.74876315e+14  -2.15675159e+13
   -3.15942425e+14  -1.32563691e+12]
 [ -9.78419303e+13  -2.72790087e+12  -3.04063659e+15  -7.26366966e+13
   -1.09418553e+15  -5.79974935e+12]
 ..., 
 [ -1.45442524e+13  -1.37270542e+11  -3.71586880e+14  -9.84906885e+12
   -1.33199275e+14  -3.49439492e+11]
 [ -1.37146160e+13  -1.13774215e+11  -3.45649841e+14  -9.21965796e+12
   -1.23988693e+14  -3.12132993e+11]
 [ -2.65132588e+13  -1.84274813e+11  -7.12927661e+14  -1.77109834e+13
   -2.57518807e+14  -1.01314586e+12]]
Loss
-1.11359e+13
Action prob
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [9]:
env = VREPPushTaskEnvironment()
states = env.reset(True)

In [16]:
del model
del policy
del env

In [17]:
del env

NameError: name 'env' is not defined

In [ ]:
K.learning_phase()

In [ ]:
states

In [ ]:
import time
for _ in range(200):
    states = env.step(generateRandomVel(2)[np.newaxis, :])
    print(states)
    #time.sleep(0.05)